### Домашнее задание №6

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Источник данных:
https://www.machinelearningmastery.ru/standard-machine-learning-datasets/

Описание данных:
**Набор данных диабета индейцев Пима**

Набор данных о диабете индейцев пима включает прогнозирование возникновения диабета в течение 5 лет у индейцев пима с учетом медицинских данных.

Это бинарная (2-классная) задача классификации. Количество наблюдений для каждого класса не сбалансировано. Есть 768 наблюдений с 8 входными переменными и 1 выходной переменной. Считается, что пропущенные значения кодируются нулевыми значениями. Имена переменных следующие:

    Количество раз беременных.
    Концентрация глюкозы в плазме через 2 часа при оральном тесте на толерантность к глюкозе.
    Диастолическое артериальное давление (мм рт. Ст.).
    Толщина трехглавой кожной складки (мм).
    2-часовой сывороточный инсулин (мю Ед / мл).
    Индекс массы тела (вес в кг / (рост в м) ^ 2).
    Родословная функция диабета.
    Возраст (годы).
    Переменная класса (0 или 1).

Базовая эффективность прогнозирования наиболее распространенного класса - точность классификации приблизительно 65%. Лучшие результаты достигают точности классификации примерно 77%.

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('pima-indians-diabetes.data.csv', header=None)
data.head(3)

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


У нас есть 8 признаков и 1 целевая переменная (бинарная) - нужно спрогнозировать возникновения диабета в течение 5 лет

In [2]:
print(data.shape)

(768, 9)


Всего 768 наблюдений

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


датасет представлен числовыми наблюдениями (целыми и вещественными)

In [4]:
data.isna().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64

пропусков нет

Посмотрим на соотношение классов

In [5]:
data.iloc[:, -1].value_counts()

0    500
1    268
Name: 8, dtype: int64

классы несколько разбалансированы

__Создадим датафрейм результатов использования различных моделей:__

In [6]:
models_results = {
    'Model name': [],
    'f1': [],
    'roc': [],
    'recall': [],
    'precision': []  
}

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [7]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=7)

In [8]:
import xgboost as xgb
model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

[18:14:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [9]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    models_results['f1'].append(f1)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    models_results['roc'].append(roc)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    models_results['recall'].append(rec)
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    models_results['precision'].append(prc)
    print("precision: %.2f%%" % (prc * 100.0)) 

models_results['Model name'].append('xgboost')   
evaluate_results(y_test, y_predict)

Classification results:
f1: 61.25%
roc: 69.98%
recall: 58.33%
precision: 64.47%


In [10]:
import lightgbm as lgbm
model_lgbm = lgbm.LGBMClassifier(random_state=21)
model_lgbm.fit(x_train, y_train)

y_predict = model_lgbm.predict(x_test)

Проверяем качество

In [11]:
models_results['Model name'].append('lightgbm')  
evaluate_results(y_test, y_predict)

Classification results:
f1: 64.24%
roc: 72.02%
recall: 63.10%
precision: 65.43%


In [12]:
import catboost as catb
model_catb = catb.CatBoostClassifier(silent=True, random_state=21)
model_catb.fit(x_train, y_train)

y_predict = model_catb.predict(x_test)

In [13]:
models_results['Model name'].append('catboost')  
evaluate_results(y_test, y_predict)

Classification results:
f1: 69.51%
roc: 76.11%
recall: 67.86%
precision: 71.25%


Интересно, но "из коробки" лючший результат у алгоритма catboost

### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [14]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
k = 0.25
pos_sample_len = int(np.ceil(k * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 67/268 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [15]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    701
 1     67
Name: class_test, dtype: int64


In [16]:
mod_data.head(10)

,0,1,2,3,4,5,6,7,8,class_test
0,6,148,72,35,0,33.6,0.627,50,1,1
1,1,85,66,29,0,26.6,0.351,31,0,-1
2,8,183,64,0,0,23.3,0.672,32,1,-1
3,1,89,66,23,94,28.1,0.167,21,0,-1
4,0,137,40,35,168,43.1,2.288,33,1,-1
5,5,116,74,0,0,25.6,0.201,30,0,-1
6,3,78,50,32,88,31.0,0.248,26,1,1
7,10,115,0,0,0,35.3,0.134,29,0,-1
8,2,197,70,45,543,30.5,0.158,53,1,-1
9,8,125,96,0,0,0.0,0.232,54,1,-1


In [17]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling

In [18]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(67, 10) (67, 10)


In [19]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

models_results['Model name'].append(f'PU learning_{k}_xgboost') 
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

[18:14:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 54.34%
roc: 66.99%
recall: 79.12%
precision: 41.38%


C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [20]:
model_lgbm = lgbm.LGBMClassifier(random_state=21)
model_lgbm.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)

y_predict = model_lgbm.predict(sample_test.iloc[:,:-2].values)

models_results['Model name'].append(f'PU learning_{k}_lightgbm') 
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 55.20%
roc: 67.88%
recall: 80.22%
precision: 42.07%


In [21]:
model_catb = catb.CatBoostClassifier(silent=True, random_state=21)
model_catb.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)

y_predict = model_catb.predict(sample_test.iloc[:,:-2].values)

models_results['Model name'].append(f'PU learning_{k}_catboost') 
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 56.32%
roc: 68.96%
recall: 85.71%
precision: 41.94%


In [22]:
pd.DataFrame(data=models_results).sort_values('f1', ascending=False)

,Model name,f1,roc,recall,precision
2,catboost,0.695122,0.761054,0.678571,0.712500
1,lightgbm,0.642424,0.720238,0.630952,0.654321
0,xgboost,0.612500,0.699830,0.583333,0.644737
5,PU learning_0.25_catboost,0.563177,0.689633,0.857143,0.419355
4,PU learning_0.25_lightgbm,0.551985,0.678754,0.802198,0.420749
3,PU learning_0.25_xgboost,0.543396,0.669941,0.791209,0.413793


Поэкспериментируем с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [23]:
k = [0.1, 0.3, 0.4, 0.5, 0.6, 0.7]

In [24]:
for x in k:
    mod_data = data.copy()
    #get the indices of the positives samples
    pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
    #shuffle them
    np.random.shuffle(pos_ind)
    # leave just 25% of the positives marked
    pos_sample_len = int(np.ceil(x * len(pos_ind)))
    print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
    pos_sample = pos_ind[:pos_sample_len]
    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1
    print('target variable:\n', mod_data.iloc[:,-1].value_counts())
    x_data = mod_data.iloc[:,:-2].values # just the X 
    y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
    y_positive = mod_data.iloc[:,-2].values # original class
    mod_data = mod_data.sample(frac=1)
    neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
    sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
    pos_sample = mod_data[mod_data['class_test']==1]
    print(neg_sample.shape, pos_sample.shape)
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
    model = xgb.XGBClassifier()
    model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
    y_predict = model.predict(sample_test.iloc[:,:-2].values)
    models_results['Model name'].append(f'PU learning_{x}_xgboost') 
    evaluate_results(sample_test.iloc[:,-2].values, y_predict)
    model_lgbm = lgbm.LGBMClassifier(random_state=21)
    model_lgbm.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
    y_predict = model_lgbm.predict(sample_test.iloc[:,:-2].values)
    models_results['Model name'].append(f'PU learning_{x}_lightgbm') 
    evaluate_results(sample_test.iloc[:,-2].values, y_predict)
    model_catb = catb.CatBoostClassifier(silent=True, random_state=21)
    model_catb.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
    y_predict = model_catb.predict(sample_test.iloc[:,:-2].values)
    models_results['Model name'].append(f'PU learning_{x}_catboost') 
    evaluate_results(sample_test.iloc[:,-2].values, y_predict)    

Using 27/268 as positives and unlabeling the rest
target variable:
 -1    741
 1     27
Name: class_test, dtype: int64
(27, 10) (27, 10)
[18:14:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 52.84%
roc: 61.68%
recall: 77.97%
precision: 39.95%
Classification results:
f1: 53.80%
roc: 62.50%
recall: 81.06%
precision: 40.26%


C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Classification results:
f1: 57.95%
roc: 66.83%
recall: 90.75%
precision: 42.56%
Using 81/268 as positives and unlabeling the rest
target variable:
 -1    687
 1     81
Name: class_test, dtype: int64
(81, 10) (81, 10)
[18:14:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 56.80%
roc: 72.39%
recall: 89.31%
precision: 41.64%
Classification results:
f1: 56.29%
roc: 71.86%
recall: 88.68%
precision: 41.23%


C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Classification results:
f1: 57.03%
roc: 72.83%
recall: 93.08%
precision: 41.11%
Using 108/268 as positives and unlabeling the rest
target variable:
 -1    660
 1    108
Name: class_test, dtype: int64
(108, 10) (108, 10)
[18:14:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 49.52%
roc: 67.96%
recall: 80.00%
precision: 35.86%
Classification results:
f1: 49.65%
roc: 68.14%
recall: 81.54%
precision: 35.69%


...

Classification results:
f1: 51.92%
roc: 73.75%
recall: 84.62%
precision: 37.45%
Using 161/268 as positives and unlabeling the rest
target variable:
 -1    607
 1    161
Name: class_test, dtype: int64
(161, 10) (161, 10)
[18:14:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 50.00%
roc: 72.79%
recall: 78.16%
precision: 36.76%
Classification results:
f1: 51.47%
roc: 74.21%
recall: 80.46%
precision: 37.84%


C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Classification results:
f1: 48.75%
roc: 71.81%
recall: 78.16%
precision: 35.42%
Using 188/268 as positives and unlabeling the rest
target variable:
 -1    580
 1    188
Name: class_test, dtype: int64
(188, 10) (188, 10)
[18:14:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 36.75%
roc: 70.90%
recall: 82.69%
precision: 23.63%
Classification results:
f1: 38.30%
roc: 72.98%
recall: 86.54%
precision: 24.59%


C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Mikhail\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


Classification results:
f1: 40.72%
roc: 75.03%
recall: 86.54%
precision: 26.63%


In [25]:
for x in list(models_results.keys())[1:]:
    print(x)
    print(pd.DataFrame(data=models_results).sort_values(x, ascending=False).head(3))

f1
  Model name        f1       roc    recall  precision
2   catboost  0.695122  0.761054  0.678571   0.712500
1   lightgbm  0.642424  0.720238  0.630952   0.654321
0    xgboost  0.612500  0.699830  0.583333   0.644737
roc
                  Model name        f1       roc    recall  precision
2                   catboost  0.695122  0.761054  0.678571   0.712500
23  PU learning_0.7_catboost  0.407240  0.750339  0.865385   0.266272
19  PU learning_0.6_lightgbm  0.514706  0.742132  0.804598   0.378378
recall
                  Model name        f1       roc    recall  precision
11  PU learning_0.3_catboost  0.570328  0.728272  0.930818   0.411111
8   PU learning_0.1_catboost  0.579466  0.668324  0.907489   0.425620
9    PU learning_0.3_xgboost  0.568000  0.723946  0.893082   0.416422
precision
  Model name        f1       roc    recall  precision
2   catboost  0.695122  0.761054  0.678571   0.712500
1   lightgbm  0.642424  0.720238  0.630952   0.654321
0    xgboost  0.612500  0.699830  0.58

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

Ваш ответ здесь:

_Я думаю, что более популярен random negative sampling, главным образом благодаря тому, что проблема в итоге сводится к задаче бинарной классификации_